# Начальная инициализация

Установить Pytorch https://pytorch.org/

In [1]:
#импорт библиотек
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
# import scipy
import torch.utils.data as data_utils
from sklearn.metrics import classification_report, f1_score

from sklearn import datasets

import torch.optim as optim

import pandas as pd
import numpy as np

In [2]:
# Подключение вычислений на видеокарту, если доступна
if torch.cuda.is_available(): 
 dev = "cuda:0" 
else: 
 dev = "cpu" 
device = torch.device(dev)
device = "cpu"

In [3]:
print(device)

cpu


# Задание 1

## Сеть c 2 скрытыми слоями, разные слои, lr=0.01

In [4]:
X_train = pd.read_csv('X_train_Final.csv').values
y_train = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_train_Final.csv', names=['class']).values.tolist()])
X_test = pd.read_csv('X_test_Final.csv').values
y_test = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_test_Final.csv', names=['class']).values.tolist()])

In [5]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(20, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 4)
        self.fc4 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=20, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=4, bias=True)
  (fc4): Linear(in_features=4, out_features=2, bias=True)
)


In [6]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [7]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

In [8]:
for epoch in range(15): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.7058, grad_fn=<NllLossBackward0>)
tensor(0.5969, grad_fn=<NllLossBackward0>)
tensor(0.5517, grad_fn=<NllLossBackward0>)
tensor(0.6002, grad_fn=<NllLossBackward0>)
tensor(0.5904, grad_fn=<NllLossBackward0>)
tensor(0.6249, grad_fn=<NllLossBackward0>)
tensor(0.6038, grad_fn=<NllLossBackward0>)
tensor(0.7043, grad_fn=<NllLossBackward0>)
tensor(0.5648, grad_fn=<NllLossBackward0>)
tensor(0.8286, grad_fn=<NllLossBackward0>)
tensor(0.7759, grad_fn=<NllLossBackward0>)
tensor(0.6638, grad_fn=<NllLossBackward0>)
tensor(0.7769, grad_fn=<NllLossBackward0>)
tensor(0.7274, grad_fn=<NllLossBackward0>)
tensor(0.7833, grad_fn=<NllLossBackward0>)


In [9]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [10]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([1, 2])
50
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00        23
    positive       0.54      1.00      0.70        27

    accuracy                           0.54        50
   macro avg       0.27      0.50      0.35        50
weighted avg       0.29      0.54      0.38        50

f1 = 0.7012987012987013


/home/max/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/max/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/max/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Сеть c 2 скрытыми слоями, разные слои, lr=0.001

In [11]:
X_train = pd.read_csv('X_train_Final.csv').values
y_train = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_train_Final.csv', names=['class']).values.tolist()])
X_test = pd.read_csv('X_test_Final.csv').values
y_test = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_test_Final.csv', names=['class']).values.tolist()])

In [12]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(20, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 4)
        self.fc4 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=20, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=4, bias=True)
  (fc4): Linear(in_features=4, out_features=2, bias=True)
)


In [13]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [14]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

In [15]:
for epoch in range(15): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.5383, grad_fn=<NllLossBackward0>)
tensor(0.5882, grad_fn=<NllLossBackward0>)
tensor(0.6179, grad_fn=<NllLossBackward0>)
tensor(0.9777, grad_fn=<NllLossBackward0>)
tensor(0.7616, grad_fn=<NllLossBackward0>)
tensor(0.6337, grad_fn=<NllLossBackward0>)
tensor(0.2905, grad_fn=<NllLossBackward0>)
tensor(0.5513, grad_fn=<NllLossBackward0>)
tensor(0.7236, grad_fn=<NllLossBackward0>)
tensor(0.8097, grad_fn=<NllLossBackward0>)
tensor(0.7692, grad_fn=<NllLossBackward0>)
tensor(0.6888, grad_fn=<NllLossBackward0>)
tensor(0.3534, grad_fn=<NllLossBackward0>)
tensor(0.6768, grad_fn=<NllLossBackward0>)
tensor(0.4462, grad_fn=<NllLossBackward0>)


In [16]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [17]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([1, 2])
50
              precision    recall  f1-score   support

    negative       0.75      0.39      0.51        23
    positive       0.63      0.89      0.74        27

    accuracy                           0.66        50
   macro avg       0.69      0.64      0.63        50
weighted avg       0.69      0.66      0.64        50

f1 = 0.7384615384615385


## Сеть c 2 скрытыми слоями, разные слои, lr=0.0001

In [18]:
X_train = pd.read_csv('X_train_Final.csv').values
y_train = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_train_Final.csv', names=['class']).values.tolist()])
X_test = pd.read_csv('X_test_Final.csv').values
y_test = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_test_Final.csv', names=['class']).values.tolist()])

In [19]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(20, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 4)
        self.fc4 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=20, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=4, bias=True)
  (fc4): Linear(in_features=4, out_features=2, bias=True)
)


In [20]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [21]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

In [22]:
for epoch in range(15): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.5984, grad_fn=<NllLossBackward0>)
tensor(0.6093, grad_fn=<NllLossBackward0>)
tensor(0.7532, grad_fn=<NllLossBackward0>)
tensor(0.7883, grad_fn=<NllLossBackward0>)
tensor(0.6592, grad_fn=<NllLossBackward0>)
tensor(0.5823, grad_fn=<NllLossBackward0>)
tensor(0.5911, grad_fn=<NllLossBackward0>)
tensor(0.7203, grad_fn=<NllLossBackward0>)
tensor(0.5463, grad_fn=<NllLossBackward0>)
tensor(0.9425, grad_fn=<NllLossBackward0>)
tensor(0.6866, grad_fn=<NllLossBackward0>)
tensor(0.4436, grad_fn=<NllLossBackward0>)
tensor(0.4812, grad_fn=<NllLossBackward0>)
tensor(0.6286, grad_fn=<NllLossBackward0>)
tensor(0.4227, grad_fn=<NllLossBackward0>)


In [23]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [24]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([1, 2])
50
              precision    recall  f1-score   support

    negative       0.72      0.57      0.63        23
    positive       0.69      0.81      0.75        27

    accuracy                           0.70        50
   macro avg       0.70      0.69      0.69        50
weighted avg       0.70      0.70      0.69        50

f1 = 0.7457627118644067


## Сеть c 2 скрытыми слоями, одинаковые слои, lr=0.01

In [25]:
X_train = pd.read_csv('X_train_Final.csv').values
y_train = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_train_Final.csv', names=['class']).values.tolist()])
X_test = pd.read_csv('X_test_Final.csv').values
y_test = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_test_Final.csv', names=['class']).values.tolist()])

In [26]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(20, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=20, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=2, bias=True)
)


In [27]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [28]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

In [29]:
for epoch in range(15): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6592, grad_fn=<NllLossBackward0>)
tensor(0.7356, grad_fn=<NllLossBackward0>)
tensor(0.7149, grad_fn=<NllLossBackward0>)
tensor(0.3831, grad_fn=<NllLossBackward0>)
tensor(1.2606, grad_fn=<NllLossBackward0>)
tensor(0.5238, grad_fn=<NllLossBackward0>)
tensor(0.7455, grad_fn=<NllLossBackward0>)
tensor(0.4544, grad_fn=<NllLossBackward0>)
tensor(0.6935, grad_fn=<NllLossBackward0>)
tensor(0.4782, grad_fn=<NllLossBackward0>)
tensor(0.3551, grad_fn=<NllLossBackward0>)
tensor(0.3507, grad_fn=<NllLossBackward0>)
tensor(0.8327, grad_fn=<NllLossBackward0>)
tensor(0.3346, grad_fn=<NllLossBackward0>)
tensor(0.4552, grad_fn=<NllLossBackward0>)


In [30]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [31]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([1, 2])
50
              precision    recall  f1-score   support

    negative       0.72      0.78      0.75        23
    positive       0.80      0.74      0.77        27

    accuracy                           0.76        50
   macro avg       0.76      0.76      0.76        50
weighted avg       0.76      0.76      0.76        50

f1 = 0.7692307692307692


## Сеть c 2 скрытыми слоями, одинаковые слои, lr=0.001

In [32]:
X_train = pd.read_csv('X_train_Final.csv').values
y_train = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_train_Final.csv', names=['class']).values.tolist()])
X_test = pd.read_csv('X_test_Final.csv').values
y_test = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_test_Final.csv', names=['class']).values.tolist()])

In [33]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(20, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=20, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=2, bias=True)
)


In [34]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [35]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

In [36]:
for epoch in range(15): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.5702, grad_fn=<NllLossBackward0>)
tensor(0.5496, grad_fn=<NllLossBackward0>)
tensor(1.0997, grad_fn=<NllLossBackward0>)
tensor(0.4017, grad_fn=<NllLossBackward0>)
tensor(0.5625, grad_fn=<NllLossBackward0>)
tensor(0.7751, grad_fn=<NllLossBackward0>)
tensor(0.2544, grad_fn=<NllLossBackward0>)
tensor(0.3497, grad_fn=<NllLossBackward0>)
tensor(0.5995, grad_fn=<NllLossBackward0>)
tensor(0.4259, grad_fn=<NllLossBackward0>)
tensor(0.6130, grad_fn=<NllLossBackward0>)
tensor(0.6986, grad_fn=<NllLossBackward0>)
tensor(0.3885, grad_fn=<NllLossBackward0>)
tensor(0.5264, grad_fn=<NllLossBackward0>)
tensor(0.7515, grad_fn=<NllLossBackward0>)


In [37]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [38]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([1, 2])
50
              precision    recall  f1-score   support

    negative       0.79      0.65      0.71        23
    positive       0.74      0.85      0.79        27

    accuracy                           0.76        50
   macro avg       0.77      0.75      0.75        50
weighted avg       0.76      0.76      0.76        50

f1 = 0.7931034482758621


## Сеть c 2 скрытыми слоями, одинаковые слои, lr=0.0001

In [39]:
X_train = pd.read_csv('X_train_Final.csv').values
y_train = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_train_Final.csv', names=['class']).values.tolist()])
X_test = pd.read_csv('X_test_Final.csv').values
y_test = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_test_Final.csv', names=['class']).values.tolist()])

In [40]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(20, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=20, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=2, bias=True)
)


In [41]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [42]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

In [43]:
for epoch in range(15): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6574, grad_fn=<NllLossBackward0>)
tensor(0.7843, grad_fn=<NllLossBackward0>)
tensor(0.6048, grad_fn=<NllLossBackward0>)
tensor(0.8009, grad_fn=<NllLossBackward0>)
tensor(0.6763, grad_fn=<NllLossBackward0>)
tensor(0.5091, grad_fn=<NllLossBackward0>)
tensor(0.3542, grad_fn=<NllLossBackward0>)
tensor(0.5508, grad_fn=<NllLossBackward0>)
tensor(1.0533, grad_fn=<NllLossBackward0>)
tensor(0.7505, grad_fn=<NllLossBackward0>)
tensor(0.8456, grad_fn=<NllLossBackward0>)
tensor(0.4631, grad_fn=<NllLossBackward0>)
tensor(0.4595, grad_fn=<NllLossBackward0>)
tensor(0.4857, grad_fn=<NllLossBackward0>)
tensor(0.8733, grad_fn=<NllLossBackward0>)


In [44]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [45]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([1, 2])
50
              precision    recall  f1-score   support

    negative       0.75      0.65      0.70        23
    positive       0.73      0.81      0.77        27

    accuracy                           0.74        50
   macro avg       0.74      0.73      0.73        50
weighted avg       0.74      0.74      0.74        50

f1 = 0.7719298245614035


## Сеть c 3 скрытыми слоями, разные слои, lr=0.01

In [46]:
X_train = pd.read_csv('X_train_Final.csv').values
y_train = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_train_Final.csv', names=['class']).values.tolist()])
X_test = pd.read_csv('X_test_Final.csv').values
y_test = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_test_Final.csv', names=['class']).values.tolist()])

In [47]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(20, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 4)
        self.fc5 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=20, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=4, bias=True)
  (fc5): Linear(in_features=4, out_features=2, bias=True)
)


In [48]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [49]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

In [50]:
for epoch in range(15): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6438, grad_fn=<NllLossBackward0>)
tensor(0.7732, grad_fn=<NllLossBackward0>)
tensor(0.8834, grad_fn=<NllLossBackward0>)
tensor(1.1941, grad_fn=<NllLossBackward0>)
tensor(0.5586, grad_fn=<NllLossBackward0>)
tensor(0.6656, grad_fn=<NllLossBackward0>)
tensor(0.3545, grad_fn=<NllLossBackward0>)
tensor(1.2170, grad_fn=<NllLossBackward0>)
tensor(0.7411, grad_fn=<NllLossBackward0>)
tensor(0.7073, grad_fn=<NllLossBackward0>)
tensor(0.2350, grad_fn=<NllLossBackward0>)
tensor(0.8363, grad_fn=<NllLossBackward0>)
tensor(0.1166, grad_fn=<NllLossBackward0>)
tensor(1.1427, grad_fn=<NllLossBackward0>)
tensor(0.7229, grad_fn=<NllLossBackward0>)


In [51]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [52]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([1, 2])
50
              precision    recall  f1-score   support

    negative       0.69      0.39      0.50        23
    positive       0.62      0.85      0.72        27

    accuracy                           0.64        50
   macro avg       0.66      0.62      0.61        50
weighted avg       0.65      0.64      0.62        50



## Сеть c 3 скрытыми слоями, разные слои, lr=0.001

In [53]:
X_train = pd.read_csv('X_train_Final.csv').values
y_train = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_train_Final.csv', names=['class']).values.tolist()])
X_test = pd.read_csv('X_test_Final.csv').values
y_test = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_test_Final.csv', names=['class']).values.tolist()])

In [54]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(20, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 4)
        self.fc5 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=20, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=4, bias=True)
  (fc5): Linear(in_features=4, out_features=2, bias=True)
)


In [55]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [56]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

In [57]:
for epoch in range(15): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.4060, grad_fn=<NllLossBackward0>)
tensor(0.3196, grad_fn=<NllLossBackward0>)
tensor(0.3433, grad_fn=<NllLossBackward0>)
tensor(0.3873, grad_fn=<NllLossBackward0>)
tensor(0.4286, grad_fn=<NllLossBackward0>)
tensor(0.5999, grad_fn=<NllLossBackward0>)
tensor(0.7312, grad_fn=<NllLossBackward0>)
tensor(0.2862, grad_fn=<NllLossBackward0>)
tensor(0.5396, grad_fn=<NllLossBackward0>)
tensor(0.6180, grad_fn=<NllLossBackward0>)
tensor(0.6753, grad_fn=<NllLossBackward0>)
tensor(1.0316, grad_fn=<NllLossBackward0>)
tensor(0.7847, grad_fn=<NllLossBackward0>)
tensor(0.8569, grad_fn=<NllLossBackward0>)
tensor(0.3438, grad_fn=<NllLossBackward0>)


In [58]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [59]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([1, 2])
50
              precision    recall  f1-score   support

    negative       0.76      0.70      0.73        23
    positive       0.76      0.81      0.79        27

    accuracy                           0.76        50
   macro avg       0.76      0.76      0.76        50
weighted avg       0.76      0.76      0.76        50



## Сеть c 3 скрытыми слоями, разные слои, lr=0.0001

In [60]:
X_train = pd.read_csv('X_train_Final.csv').values
y_train = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_train_Final.csv', names=['class']).values.tolist()])
X_test = pd.read_csv('X_test_Final.csv').values
y_test = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_test_Final.csv', names=['class']).values.tolist()])

In [61]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(20, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 4)
        self.fc5 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=20, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=4, bias=True)
  (fc5): Linear(in_features=4, out_features=2, bias=True)
)


In [62]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [63]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

In [64]:
for epoch in range(15): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.7999, grad_fn=<NllLossBackward0>)
tensor(0.6153, grad_fn=<NllLossBackward0>)
tensor(0.6234, grad_fn=<NllLossBackward0>)
tensor(0.7637, grad_fn=<NllLossBackward0>)
tensor(0.6215, grad_fn=<NllLossBackward0>)
tensor(0.7731, grad_fn=<NllLossBackward0>)
tensor(0.7608, grad_fn=<NllLossBackward0>)
tensor(0.6167, grad_fn=<NllLossBackward0>)
tensor(0.8075, grad_fn=<NllLossBackward0>)
tensor(0.6454, grad_fn=<NllLossBackward0>)
tensor(0.5406, grad_fn=<NllLossBackward0>)
tensor(0.5742, grad_fn=<NllLossBackward0>)
tensor(0.5078, grad_fn=<NllLossBackward0>)
tensor(0.7092, grad_fn=<NllLossBackward0>)
tensor(0.7086, grad_fn=<NllLossBackward0>)


In [65]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [66]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([1, 2])
50
              precision    recall  f1-score   support

    negative       0.68      0.65      0.67        23
    positive       0.71      0.74      0.73        27

    accuracy                           0.70        50
   macro avg       0.70      0.70      0.70        50
weighted avg       0.70      0.70      0.70        50



## Сеть c 3 скрытыми слоями, одинаковые слои, lr=0.01

In [67]:
X_train = pd.read_csv('X_train_Final.csv').values
y_train = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_train_Final.csv', names=['class']).values.tolist()])
X_test = pd.read_csv('X_test_Final.csv').values
y_test = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_test_Final.csv', names=['class']).values.tolist()])

In [68]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(20, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 32)
        self.fc4 = nn.Linear(32, 32)
        self.fc5 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=20, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=2, bias=True)
)


In [69]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [70]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

In [71]:
for epoch in range(15): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.5268, grad_fn=<NllLossBackward0>)
tensor(0.5234, grad_fn=<NllLossBackward0>)
tensor(0.8228, grad_fn=<NllLossBackward0>)
tensor(0.7991, grad_fn=<NllLossBackward0>)
tensor(1.3654, grad_fn=<NllLossBackward0>)
tensor(0.5322, grad_fn=<NllLossBackward0>)
tensor(0.4255, grad_fn=<NllLossBackward0>)
tensor(0.6885, grad_fn=<NllLossBackward0>)
tensor(0.5021, grad_fn=<NllLossBackward0>)
tensor(0.4594, grad_fn=<NllLossBackward0>)
tensor(0.4984, grad_fn=<NllLossBackward0>)
tensor(0.6432, grad_fn=<NllLossBackward0>)
tensor(0.2391, grad_fn=<NllLossBackward0>)
tensor(0.5887, grad_fn=<NllLossBackward0>)
tensor(0.3941, grad_fn=<NllLossBackward0>)


In [72]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [73]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([1, 2])
50
              precision    recall  f1-score   support

    negative       0.67      0.78      0.72        23
    positive       0.78      0.67      0.72        27

    accuracy                           0.72        50
   macro avg       0.72      0.72      0.72        50
weighted avg       0.73      0.72      0.72        50



## Сеть c 3 скрытыми слоями, одинаковые слои, lr=0.001

In [74]:
X_train = pd.read_csv('X_train_Final.csv').values
y_train = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_train_Final.csv', names=['class']).values.tolist()])
X_test = pd.read_csv('X_test_Final.csv').values
y_test = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_test_Final.csv', names=['class']).values.tolist()])

In [75]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(20, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 32)
        self.fc4 = nn.Linear(32, 32)
        self.fc5 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=20, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=2, bias=True)
)


In [76]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [77]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

In [78]:
for epoch in range(15): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.5197, grad_fn=<NllLossBackward0>)
tensor(0.8682, grad_fn=<NllLossBackward0>)
tensor(0.3669, grad_fn=<NllLossBackward0>)
tensor(0.6058, grad_fn=<NllLossBackward0>)
tensor(0.7846, grad_fn=<NllLossBackward0>)
tensor(0.9395, grad_fn=<NllLossBackward0>)
tensor(0.7246, grad_fn=<NllLossBackward0>)
tensor(0.7579, grad_fn=<NllLossBackward0>)
tensor(0.5767, grad_fn=<NllLossBackward0>)
tensor(0.1586, grad_fn=<NllLossBackward0>)
tensor(0.6291, grad_fn=<NllLossBackward0>)
tensor(0.7444, grad_fn=<NllLossBackward0>)
tensor(1.3422, grad_fn=<NllLossBackward0>)
tensor(0.5442, grad_fn=<NllLossBackward0>)
tensor(0.4959, grad_fn=<NllLossBackward0>)


In [79]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [80]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([1, 2])
50
              precision    recall  f1-score   support

    negative       0.75      0.65      0.70        23
    positive       0.73      0.81      0.77        27

    accuracy                           0.74        50
   macro avg       0.74      0.73      0.73        50
weighted avg       0.74      0.74      0.74        50



## Сеть c 3 скрытыми слоями, одинаковые слои, lr=0.0001

In [81]:
X_train = pd.read_csv('X_train_Final.csv').values
y_train = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_train_Final.csv', names=['class']).values.tolist()])
X_test = pd.read_csv('X_test_Final.csv').values
y_test = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_test_Final.csv', names=['class']).values.tolist()])

In [82]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(20, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 32)
        self.fc4 = nn.Linear(32, 32)
        self.fc5 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=20, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=2, bias=True)
)


In [83]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [84]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

In [85]:
for epoch in range(15): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6366, grad_fn=<NllLossBackward0>)
tensor(0.7517, grad_fn=<NllLossBackward0>)
tensor(0.7509, grad_fn=<NllLossBackward0>)
tensor(0.7229, grad_fn=<NllLossBackward0>)
tensor(0.6374, grad_fn=<NllLossBackward0>)
tensor(0.7308, grad_fn=<NllLossBackward0>)
tensor(0.8103, grad_fn=<NllLossBackward0>)
tensor(0.5828, grad_fn=<NllLossBackward0>)
tensor(0.6085, grad_fn=<NllLossBackward0>)
tensor(0.8786, grad_fn=<NllLossBackward0>)
tensor(0.8668, grad_fn=<NllLossBackward0>)
tensor(0.8885, grad_fn=<NllLossBackward0>)
tensor(0.7751, grad_fn=<NllLossBackward0>)
tensor(0.4826, grad_fn=<NllLossBackward0>)
tensor(0.6444, grad_fn=<NllLossBackward0>)


In [86]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [87]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([1, 2])
50
              precision    recall  f1-score   support

    negative       0.71      0.65      0.68        23
    positive       0.72      0.78      0.75        27

    accuracy                           0.72        50
   macro avg       0.72      0.71      0.72        50
weighted avg       0.72      0.72      0.72        50



## Сеть c 4 скрытыми слоями, разные слои, lr=0.01

In [88]:
X_train = pd.read_csv('X_train_Final.csv').values
y_train = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_train_Final.csv', names=['class']).values.tolist()])
X_test = pd.read_csv('X_test_Final.csv').values
y_test = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_test_Final.csv', names=['class']).values.tolist()])

In [89]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(20, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 4)
        self.fc6 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=20, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=8, bias=True)
  (fc5): Linear(in_features=8, out_features=4, bias=True)
  (fc6): Linear(in_features=4, out_features=2, bias=True)
)


In [90]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [91]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

In [92]:
for epoch in range(15): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.5507, grad_fn=<NllLossBackward0>)
tensor(0.7229, grad_fn=<NllLossBackward0>)
tensor(0.9304, grad_fn=<NllLossBackward0>)
tensor(1.0359, grad_fn=<NllLossBackward0>)
tensor(1.2731, grad_fn=<NllLossBackward0>)
tensor(0.5189, grad_fn=<NllLossBackward0>)
tensor(0.2527, grad_fn=<NllLossBackward0>)
tensor(0.4889, grad_fn=<NllLossBackward0>)
tensor(0.5770, grad_fn=<NllLossBackward0>)
tensor(0.9518, grad_fn=<NllLossBackward0>)
tensor(0.8130, grad_fn=<NllLossBackward0>)
tensor(0.4404, grad_fn=<NllLossBackward0>)
tensor(0.8898, grad_fn=<NllLossBackward0>)
tensor(0.6453, grad_fn=<NllLossBackward0>)
tensor(0.9046, grad_fn=<NllLossBackward0>)


In [93]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [94]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([1, 2])
50
              precision    recall  f1-score   support

    negative       0.66      0.83      0.73        23
    positive       0.81      0.63      0.71        27

    accuracy                           0.72        50
   macro avg       0.73      0.73      0.72        50
weighted avg       0.74      0.72      0.72        50



## Сеть c 4 скрытыми слоями, разные слои, lr=0.001

In [95]:
X_train = pd.read_csv('X_train_Final.csv').values
y_train = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_train_Final.csv', names=['class']).values.tolist()])
X_test = pd.read_csv('X_test_Final.csv').values
y_test = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_test_Final.csv', names=['class']).values.tolist()])

In [96]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(20, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 4)
        self.fc6 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=20, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=8, bias=True)
  (fc5): Linear(in_features=8, out_features=4, bias=True)
  (fc6): Linear(in_features=4, out_features=2, bias=True)
)


In [97]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [98]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

In [99]:
for epoch in range(15): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6293, grad_fn=<NllLossBackward0>)
tensor(0.6398, grad_fn=<NllLossBackward0>)
tensor(0.7693, grad_fn=<NllLossBackward0>)
tensor(0.6318, grad_fn=<NllLossBackward0>)
tensor(0.4867, grad_fn=<NllLossBackward0>)
tensor(0.6915, grad_fn=<NllLossBackward0>)
tensor(1.1857, grad_fn=<NllLossBackward0>)
tensor(0.6918, grad_fn=<NllLossBackward0>)
tensor(0.3776, grad_fn=<NllLossBackward0>)
tensor(0.6991, grad_fn=<NllLossBackward0>)
tensor(0.2970, grad_fn=<NllLossBackward0>)
tensor(0.7666, grad_fn=<NllLossBackward0>)
tensor(0.6162, grad_fn=<NllLossBackward0>)
tensor(1.0291, grad_fn=<NllLossBackward0>)
tensor(0.4862, grad_fn=<NllLossBackward0>)


In [100]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [101]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([1, 2])
50
              precision    recall  f1-score   support

    negative       0.78      0.61      0.68        23
    positive       0.72      0.85      0.78        27

    accuracy                           0.74        50
   macro avg       0.75      0.73      0.73        50
weighted avg       0.75      0.74      0.74        50



## Сеть c 4 скрытыми слоями, разные слои, lr=0.0001

In [102]:
X_train = pd.read_csv('X_train_Final.csv').values
y_train = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_train_Final.csv', names=['class']).values.tolist()])
X_test = pd.read_csv('X_test_Final.csv').values
y_test = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_test_Final.csv', names=['class']).values.tolist()])

In [103]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(20, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 4)
        self.fc6 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=20, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=8, bias=True)
  (fc5): Linear(in_features=8, out_features=4, bias=True)
  (fc6): Linear(in_features=4, out_features=2, bias=True)
)


In [104]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [105]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

In [106]:
for epoch in range(15): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6771, grad_fn=<NllLossBackward0>)
tensor(0.6404, grad_fn=<NllLossBackward0>)
tensor(0.6306, grad_fn=<NllLossBackward0>)
tensor(0.6289, grad_fn=<NllLossBackward0>)
tensor(0.6288, grad_fn=<NllLossBackward0>)
tensor(0.6271, grad_fn=<NllLossBackward0>)
tensor(0.7626, grad_fn=<NllLossBackward0>)
tensor(0.6303, grad_fn=<NllLossBackward0>)
tensor(0.6303, grad_fn=<NllLossBackward0>)
tensor(0.7579, grad_fn=<NllLossBackward0>)
tensor(0.7592, grad_fn=<NllLossBackward0>)
tensor(0.7623, grad_fn=<NllLossBackward0>)
tensor(0.7619, grad_fn=<NllLossBackward0>)
tensor(0.6312, grad_fn=<NllLossBackward0>)
tensor(0.7659, grad_fn=<NllLossBackward0>)


In [107]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [108]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([1, 2])
50
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00        23
    positive       0.54      1.00      0.70        27

    accuracy                           0.54        50
   macro avg       0.27      0.50      0.35        50
weighted avg       0.29      0.54      0.38        50



/home/max/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/max/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/max/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Сеть c 4 скрытыми слоями, одинаковые слои, lr=0.01

In [109]:
X_train = pd.read_csv('X_train_Final.csv').values
y_train = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_train_Final.csv', names=['class']).values.tolist()])
X_test = pd.read_csv('X_test_Final.csv').values
y_test = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_test_Final.csv', names=['class']).values.tolist()])

In [110]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(20, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 64)
        self.fc6 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=20, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=2, bias=True)
)


In [111]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [112]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

In [113]:
for epoch in range(15): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.7644, grad_fn=<NllLossBackward0>)
tensor(0.5935, grad_fn=<NllLossBackward0>)
tensor(0.7860, grad_fn=<NllLossBackward0>)
tensor(0.2950, grad_fn=<NllLossBackward0>)
tensor(0.8798, grad_fn=<NllLossBackward0>)
tensor(0.6768, grad_fn=<NllLossBackward0>)
tensor(0.7973, grad_fn=<NllLossBackward0>)
tensor(0.7843, grad_fn=<NllLossBackward0>)
tensor(1.1516, grad_fn=<NllLossBackward0>)
tensor(1.3142, grad_fn=<NllLossBackward0>)
tensor(0.3749, grad_fn=<NllLossBackward0>)
tensor(0.3913, grad_fn=<NllLossBackward0>)
tensor(0.3416, grad_fn=<NllLossBackward0>)
tensor(0.4739, grad_fn=<NllLossBackward0>)
tensor(0.8012, grad_fn=<NllLossBackward0>)


In [114]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [115]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([1, 2])
50
              precision    recall  f1-score   support

    negative       0.81      0.74      0.77        23
    positive       0.79      0.85      0.82        27

    accuracy                           0.80        50
   macro avg       0.80      0.80      0.80        50
weighted avg       0.80      0.80      0.80        50



## Сеть c 4 скрытыми слоями, одинаковые слои, lr=0.001

In [116]:
X_train = pd.read_csv('X_train_Final.csv').values
y_train = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_train_Final.csv', names=['class']).values.tolist()])
X_test = pd.read_csv('X_test_Final.csv').values
y_test = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_test_Final.csv', names=['class']).values.tolist()])

In [117]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(20, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 64)
        self.fc6 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=20, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=2, bias=True)
)


In [118]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [119]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

In [120]:
for epoch in range(15): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.7659, grad_fn=<NllLossBackward0>)
tensor(0.7133, grad_fn=<NllLossBackward0>)
tensor(0.7921, grad_fn=<NllLossBackward0>)
tensor(1.0575, grad_fn=<NllLossBackward0>)
tensor(0.8213, grad_fn=<NllLossBackward0>)
tensor(0.8907, grad_fn=<NllLossBackward0>)
tensor(0.4383, grad_fn=<NllLossBackward0>)
tensor(0.2820, grad_fn=<NllLossBackward0>)
tensor(0.4624, grad_fn=<NllLossBackward0>)
tensor(0.3422, grad_fn=<NllLossBackward0>)
tensor(0.7170, grad_fn=<NllLossBackward0>)
tensor(1.0356, grad_fn=<NllLossBackward0>)
tensor(0.6866, grad_fn=<NllLossBackward0>)
tensor(0.7796, grad_fn=<NllLossBackward0>)
tensor(0.7881, grad_fn=<NllLossBackward0>)


In [121]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [122]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([1, 2])
50
              precision    recall  f1-score   support

    negative       0.70      0.70      0.70        23
    positive       0.74      0.74      0.74        27

    accuracy                           0.72        50
   macro avg       0.72      0.72      0.72        50
weighted avg       0.72      0.72      0.72        50



## Сеть c 4 скрытыми слоями, одинаковые слои, lr=0.0001

In [123]:
X_train = pd.read_csv('X_train_Final.csv').values
y_train = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_train_Final.csv', names=['class']).values.tolist()])
X_test = pd.read_csv('X_test_Final.csv').values
y_test = np.array([0 if el[0] == -1 else el[0] for el in pd.read_csv('y_test_Final.csv', names=['class']).values.tolist()])

In [124]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(20, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 64)
        self.fc6 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=20, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=2, bias=True)
)


In [125]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [126]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True)
testset = torch.utils.data.DataLoader(test, shuffle=False)

In [127]:
for epoch in range(15): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6536, grad_fn=<NllLossBackward0>)
tensor(0.6315, grad_fn=<NllLossBackward0>)
tensor(0.7525, grad_fn=<NllLossBackward0>)
tensor(0.7601, grad_fn=<NllLossBackward0>)
tensor(0.6236, grad_fn=<NllLossBackward0>)
tensor(0.7513, grad_fn=<NllLossBackward0>)
tensor(0.6228, grad_fn=<NllLossBackward0>)
tensor(0.6138, grad_fn=<NllLossBackward0>)
tensor(0.6183, grad_fn=<NllLossBackward0>)
tensor(0.6201, grad_fn=<NllLossBackward0>)
tensor(0.6105, grad_fn=<NllLossBackward0>)
tensor(0.7666, grad_fn=<NllLossBackward0>)
tensor(0.5725, grad_fn=<NllLossBackward0>)
tensor(0.6720, grad_fn=<NllLossBackward0>)
tensor(0.7246, grad_fn=<NllLossBackward0>)


In [128]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [129]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([1, 2])
50
              precision    recall  f1-score   support

    negative       0.82      0.39      0.53        23
    positive       0.64      0.93      0.76        27

    accuracy                           0.68        50
   macro avg       0.73      0.66      0.64        50
weighted avg       0.72      0.68      0.65        50



# Сводная таблица

| Количество скрытых слоев | Количество нейронов | Скорость обучения | weighted F1-score |
| ---: | ---: | ---: | ---: | 
| 2 | 16-8 | 0.01 | 0.727 |
| 2 | 16-8 | 0.001 | 0.718 |
| 2 | 16-8 | 0.0001 | 0.742 |
| 2 | 16-16 | 0.01 | 0.799 |
| 2 | 16-16 | 0.001 | 0.769 |
| 2 | 16-16 | 0.0001 | 0.771 |
| 3 | 32-16-8 | 0.01 | 0.705 |
| 3 | 32-16-8 | 0.001 | 0.766 |
| 3 | 32-16-8 | 0.0001 | 0. |
| 3 | 32-32-32 | 0.01 | 0. |
| 3 | 32-32-32 | 0.001 | 0. |
| 3 | 32-32-32 | 0.0001 | 0. |
| 4 | 32-2 | 0.01 | 0. |
| 4 | 32-2 | 0.001 | 0. |
| 4 | 32-2 | 0.0001 | 0. |
| 4 | 32-2 | 0.01 | 0. |
| 4 | 32-2 | 0.001 | 0. |
| 4 | 32-2 | 0.0001 | 0. |